# Purpose

### 2022-03-29
The table with posts for v0.4.1 model was deleted. We need to re-create it so that we can get primary language, which is used for counterparts logic (DE/FR/MX to English/US).

# Imports & notebook setup

In [1]:
%load_ext google.colab.data_table
%load_ext autoreload
%autoreload 2

In [2]:
# colab auth for BigQuery & google drive
from google.colab import auth, files, drive
import sys  # need sys for mounting gdrive path

auth.authenticate_user()
print('Authenticated')

Authenticated


### Install libraries

These might be necessary to read from GCS

In [3]:
# # install subclu & libraries needed to read parquet files from GCS & spreadsheets
# #  make sure to use the [colab] `extra` because it includes colab-specific libraries
# module_path = f"{g_drive_root}/MyDrive/Colab Notebooks/subreddit_clustering_i18n/[colab]"

!pip install fsspec gcsfs

## General imports

In [12]:
# Regular Imports
import os
from datetime import datetime

from google.cloud import bigquery

from tqdm import tqdm

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib_venn import venn2_unweighted, venn3_unweighted


# Set env variable needed by some libraries to get datay from BigQuery
# os.environ['GOOGLE_CLOUD_PROJECT'] = 'data-science-prod-218515'
os.environ['GOOGLE_CLOUD_PROJECT'] = 'data-prod-165221'

# Load df with comments [no longer needed]

Colab can't handle loading all the parque files at once, so we need to loop through each file and append them to the table.

File format:
- `000000000000.parquet`
- ...
- `000000000038.parquet`


In [10]:
%%time

gs_posts_folder = 'gs://i18n-subreddit-clustering/posts/top/2021-12-14_fix'

# try to read all files fails because colab doesn't have enough RAM to load them in memory
# df_posts = pd.read_parquet(gs_posts)
# print(df_posts.shape)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


# Save table to BigQuery

NOTE: Sorting is not guaranteed in the final BigQuery table.

With `pandas`, we can "force" sorting if we set `chunksize` to a number smaller than the full df size.

ETA with `pandas` & ~9 columns:
batch = 10k
- 490k sub pairs = 3:30 minutes (3.5)
- 4.9 Million sub pairs = ~37 minutes


ETA with `bq` and ~39 columns:
- 15 million posts = **48 SECONDS**

In [ ]:
BREAK

In [19]:
!bq load \
    --source_format=PARQUET \
    --project_id=reddit-employee-datasets \
    david_bermejo.subclu_posts_top_no_geo_20211214 \
    gs://i18n-subreddit-clustering/posts/top/2021-12-14_fix/*.parquet

Waiting on bqjob_r19c3d198671266f3_0000017fd66bb970_1 ... (48s) Current status: DONE   


In [20]:
# # using pandas can take over 1 hour
# #. use `bq` CLI tool instead because we're not adding any new columns
# for i in tqdm(range(39)):
#     f_name = f"{i:0>12.0f}.parquet"
#     f_parquet = f"{gs_posts_folder}/{f_name}"
#     df_i = pd.read_parquet(f_parquet)
#     print(f"\n  {f_name} shape: {df_i.shape}")
#     (
#         df_i
#         .to_gbq(
#             destination_table='david_bermejo.subclu_posts_top_no_geo_20211214',
#             project_id='reddit-employee-datasets',
#             chunksize=100000,
#             if_exists='append'
#         )
#     )